# Dataset

This notebook is used as a first veiw on the dataset for understanding how the data can be used

## Make clean csv

We add multiple csv files for each language (train, test, dev). But there are useless information. Like the age the gender and the line said. Moreover, their is no columns for the language. So we had to créate our csv with all the needed information (id, path to the audio file, language). Here the langugage columns is our labels.

In [1]:
import polars as pl
import numpy as np
import os

In [2]:
dataset_path = "../data/common_voice_kpd/"
languages = os.listdir("../data/common_voice_kpd/")
dataset_path, languages[14]

('../data/common_voice_kpd/', 'French')

In [ ]:
with open("../data/common_voice_kpd/French/train.csv", "r", encoding="utf-16") as f:
    lines = f.readline()
    lines = lines.split()
    client_id = np.array([])
    path = np.array([])
    language = np.array([])
    while True:
        lines = f.readline()
        if not lines:
            break
        lines = lines.split()
        client_id = np.append(client_id, np.int16(lines[0]))
        path = np.append(path, f"/train/{lines[1]}/{lines[2]}")
        language = np.append(language, "French")

    df = (
        pl.DataFrame({
            "client_id": client_id,
            "path":path,
            "language": language,
        })
    )

df
        

client_id,path,language
f64,str,str
98.0,"""/train/frn_trn_sp_98/common_vo…","""French"""
539.0,"""/train/frn_trn_sp_426/common_v…","""French"""
109.0,"""/train/frn_trn_sp_109/common_v…","""French"""
1068.0,"""/train/frn_trn_sp_691/common_v…","""French"""
1036.0,"""/train/frn_trn_sp_675/common_v…","""French"""
…,…,…
150.0,"""/train/frn_trn_sp_150/common_v…","""French"""
351.0,"""/train/frn_trn_sp_331/common_v…","""French"""
24.0,"""/train/frn_trn_sp_24/common_vo…","""French"""


In [70]:
file_type = ["train","test","dev"]
for lang in languages:
    for file in file_type:
        with open(f"../data/common_voice_kpd/{lang}/{file}.csv", "r", encoding="utf-16") as f:
            lines = f.readline()
            client_id = np.array([], dtype=np.int16)
            path = np.array([])
            language = np.array([])
            while True:
                lines = f.readline()
                if not lines:
                    break
                lines = lines.split()
                client_id = np.append(client_id, np.int16(lines[0]))
                path = np.append(path, f"/{lang}/{file}/{lines[1]}/{lines[2]}")
                language = np.append(language, lang)

            df = (
                pl.DataFrame({
                    "client_id": client_id,
                    "path":path,
                    "language": language,
                })
            )

        df.write_csv(f"../data/common_voice_kpd/{lang}/{file}_clean.csv")

In [7]:
file_type = ["train_clean","test_clean","dev_clean"]
for file in file_type:
    combined_df = pl.DataFrame({
        "client_id": np.array([]),
        "path": np.array([]),
        "language": np.array([]),
    }).with_columns(
        pl.col("*").cast(pl.String)
    )
    for lang in languages:
        df = pl.read_csv(
            f"../data/common_voice_kpd/{lang}/{file}.csv"
            ).with_columns(
                pl.col("*").cast(pl.String)
            )
        combined_df = pl.concat([combined_df, df])
    combined_df.write_csv(f"../data/{file}.csv")

In [40]:
dfTrain = pl.read_csv("../data/train_clean.csv").sample(fraction=1.0, shuffle=True)
dfTrain[0,1]

'/Swedish/train/swd_trn_sp_80/common_voice_sv-SE_19518866.wav'

## Load wav files

In order to create the model we had to read the data (WAV)

In [42]:
import torchaudio as ta

waveform, sample_rate = ta.load("../data/common_voice_kpd/Swedish/train/swd_trn_sp_80/common_voice_sv-SE_19518866.wav")
waveform, sample_rate

OSError: [WinError 1114] Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué. Error loading "c:\Python311\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.